In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# tf.enable_eager_execution()

In [2]:
df=pd.read_csv("fr1.csv")
X=df['X'].to_numpy()
Y=df['Y'].to_numpy()
Z=df['Z'].to_numpy()
P=df['Page'].to_numpy()

inputs_tensor = tf.convert_to_tensor([[X[i],Y[i],Z[i]] for i in range(len(X))])
outputs_tensor = tf.convert_to_tensor([[P[i]] for i in range(len(P))])
print(inputs_tensor)
print(outputs_tensor)

tf.Tensor(
[[   0.  147.   18.]
 [   0.  297.   18.]
 [   0.  447.   18.]
 ...
 [1167.  867.  153.]
 [1167. 1017.  153.]
 [1167. 1167.  153.]], shape=(14979, 3), dtype=float64)
tf.Tensor(
[[7]
 [7]
 [7]
 ...
 [1]
 [1]
 [1]], shape=(14979, 1), dtype=int32)


In [3]:
from tensorflow import keras

def pde(tape, x, f):
    df_x = tape.gradient(f, x)[:, 0:1]
    df_y = tape.gradient(f, x)[:, 1:2]
    df_z = tape.gradient(f, x)[:, 2:3]
    df_xx = tape.gradient(df_x, x)[:, 0:1]
    df_yy = tape.gradient(df_y, x)[:, 1:2]
    df_zz = tape.gradient(df_z, x)[:, 2:3]
    return tf.math.abs(df_xx + df_yy + df_zz)

Oadam = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    name='Adam'
)

class PINN(keras.models.Sequential):
    def __init__(self, pde, rate=0.00001):
        super(PINN, self).__init__()
        self.rate = rate
        self.pde = pde
    def call(self, inputs, training=False):
        if training:
            with tf.GradientTape(persistent=True) as tape:
                tape.watch(inputs)
                f_predict = super(PINN, self).call(inputs)
                self.add_loss(tf.math.multiply(self.rate , tf.reduce_mean(self.pde(tape, inputs, f_predict))))
        return super(PINN, self).call(inputs)

mypinn = PINN(pde)
mypinn.add(tf.keras.layers.Dense(64, input_shape=(3,), activation="relu"))
mypinn.add(tf.keras.layers.Dense(64, activation="relu"))
mypinn.add(tf.keras.layers.Dense(128, activation="relu"))
mypinn.add(tf.keras.layers.Dense(512, activation="relu"))
mypinn.add(tf.keras.layers.Dense(256, activation="relu"))
mypinn.add(tf.keras.layers.Dense(1))

mypinn.summary()
mypinn.compile(optimizer=Oadam, loss='mse')



Model: "pinn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 210,369
Trainable params: 210,369
Non-trainable params: 0
________________________________________________________

In [4]:
mypinn.fit(inputs_tensor,outputs_tensor, epochs=20, steps_per_epoch=300)

Train on 14979 samples
Epoch 1/20
14979/14979 [==============================] - 4s 239us/sample - loss: 83.2500
Epoch 2/20
14979/14979 [==============================] - 1s 88us/sample - loss: 3.4652
Epoch 3/20
14979/14979 [==============================] - 1s 89us/sample - loss: 3.0829
Epoch 4/20
14979/14979 [==============================] - 1s 88us/sample - loss: 2.5745
Epoch 5/20
14979/14979 [==============================] - 1s 88us/sample - loss: 1.8726
Epoch 6/20
14979/14979 [==============================] - 1s 89us/sample - loss: 1.2033
Epoch 7/20
14979/14979 [==============================] - 1s 88us/sample - loss: 0.5624
Epoch 8/20
14979/14979 [==============================] - 1s 88us/sample - loss: 0.4266
Epoch 9/20
14979/14979 [==============================] - 1s 98us/sample - loss: 0.4198
Epoch 10/20
14979/14979 [==============================] - 1s 94us/sample - loss: 0.2708
Epoch 11/20
14979/14979 [==============================] - 1s 95us/sample - loss: 0.2760
Epoch

In [5]:
weights = mypinn.get_weights()
print(weights)
np.save("weights.npy", weights, allow_pickle=True)
# model2 = tf.keras.Sequential()
# model2.add(tf.keras.layers.Dense(32, input_shape=(3,), activation="relu"))
# model2.add(tf.keras.layers.Dense(64, activation="relu"))
# model2.add(tf.keras.layers.Dense(128, activation="relu"))
# model2.add(tf.keras.layers.Dense(1))

# model2.summary()

# model2.compile(optimizer=Oadam, loss='mse')
# model2.set_weights(weights)

[array([[-0.31606522,  0.2761027 , -0.00837757,  0.10732755,  0.11538139,
        -0.10724518, -0.01623259, -0.04688534,  0.04044603,  0.16556732,
         0.23720016, -0.04903708,  0.00499604,  0.20887555,  0.23600112,
        -0.01635692, -0.2201248 ,  0.24090531, -0.04622166, -0.05966163,
         0.08499634, -0.05561122, -0.00211396,  0.05996328,  0.03675808,
         0.048414  ,  0.26904175,  0.22995801, -0.19591711, -0.04952626,
         0.23557033, -0.21364866, -0.16833478,  0.08156053, -0.0137687 ,
         0.2708022 , -0.21670039,  0.10897832,  0.26325887, -0.06426334,
         0.2380631 ,  0.26684293,  0.07795434, -0.08637242,  0.16427189,
         0.09528801,  0.02320802, -0.00356093, -0.08234317,  0.26861852,
         0.09791744, -0.11358704,  0.25125486,  0.02578283,  0.28772104,
         0.24412698, -0.15288006,  0.23224506, -0.00305308,  0.21902272,
        -0.17779173, -0.14650375,  0.01602009, -0.10927896],
       [-0.02837818,  0.00678224, -0.19252357,  0.20056143,  0

In [6]:
# # SIZE = [[166, 0, 101], [485, 432, 271]]
# SIZE = [[0, 0, 0], [488, 969, 935]]
# value_np = mypinn.predict(tf.convert_to_tensor([[x, y, SIZE[0][2]] for y in range(SIZE[0][1], SIZE[1][1] + 1) for x in range(SIZE[0][0], SIZE[1][0] + 1)]))
# # value_np = model2.predict(tf.convert_to_tensor([[x, y, SIZE[0][2]] for y in range(SIZE[0][1], SIZE[1][1] + 1) for x in range(SIZE[0][0], SIZE[1][0] + 1)]))
# for z in range(SIZE[0][2] + 1, SIZE[1][2] + 1):
#     print(z)
#     space_tensor = tf.convert_to_tensor([[x, y, z] for y in range(SIZE[0][1], SIZE[1][1] + 1) for x in range(SIZE[0][0], SIZE[1][0] + 1)])
#     # value_np = np.concatenate((value_np, mypinn.predict(space_tensor)), axis=0)
#     value_np = np.concatenate((value_np, model2.predict(space_tensor)), axis=0)

In [7]:
# value_np = value_np.reshape((SIZE[1][2] - SIZE[0][2] + 1, SIZE[1][1] - SIZE[0][1] + 1, SIZE[1][0] - SIZE[0][0] + 1))

In [8]:
# print(value_np.shape)
# np.save("value_lp_1.npy", value_np)

In [9]:
# import numpy as np

# value_np = np.load("value_lp_1.npy")

# from mayavi import mlab

# obj = mlab.contour3d( value_np, contours= 10, transparent = True)

# mlab.show()